In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium
from scipy.stats import pearsonr
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime
import calendar
from matplotlib.ticker import FuncFormatter
import datetime
import joblib
import xgboost as xgb
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
import statsmodels
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [2]:
flow = pd.read_excel('Kay Bailey.xlsx')

flow.head()

,Year,Month,Date,Airport Wells,Gal Units1,Total Flow1,Ft. Bliss Wells,Gal Units2,Total Flow2,Concentrate,Gal Units3,Total Flow3,Finished Water,Gal Units4,Total Flow4,CL2
0,2018,January,1,752138,7226,7.226,667672,2288,2.288,5180688,1276.0,1.276,577652,8365,8.365,1.34
1,2018,January,2,759308,7170,7.170,669948,2276,2.276,5181946,1258.0,1.258,585965,8313,8.313,1.31
2,2018,January,3,766545,7237,7.237,672235,2287,2.287,5183231,1285.0,1.285,594306,8341,8.341,1.31
3,2018,January,4,773753,7208,7.208,674512,2277,2.277,5184502,1271.0,1.271,602624,8318,8.318,1.32
4,2018,January,5,780945,7192,7.192,676795,2283,2.283,5185769,1267.0,1.267,610943,8319,8.319,1.31


In [3]:
column_data_types = flow.dtypes

# Print the data types
print("Data types of each column in the in predictors DataFrame:")
print(column_data_types)

Data types of each column in the in predictors DataFrame:
Year                  int64
Month                object
Date                  int64
Airport Wells         int64
Gal Units1            int64
Total Flow1         float64
Ft. Bliss Wells       int64
Gal Units2            int64
Total Flow2         float64
Concentrate           int64
Gal Units3          float64
Total Flow3         float64
Finished Water       object
Gal Units4            int64
Total Flow4         float64
CL2                 float64
dtype: object


In [4]:
energy= pd.read_excel('KBH 071923 RQT 2.xlsx')

energy.head()

,Year,Month,KWH,Billed $,$/KWH,Water Kgal,KWH/Kgal,E Billed per Kgal
0,2017,Jan,408581.0,30450.81,0.074528,130239.0,3.137163,0.233807
1,2017,Feb,384347.0,28710.77,0.074700,118555.0,3.241930,0.242173
2,2017,Mar,637308.0,46868.25,0.073541,195349.0,3.262407,0.239921
3,2017,Apr,769057.0,56103.62,0.072951,217626.0,3.533847,0.257798
4,2017,May,794757.0,58179.68,0.073204,271849.0,2.923524,0.214015


In [5]:
column_data_types = energy.dtypes

# Print the data types
print("Data types of each column in the in predictors DataFrame:")
print(column_data_types)

Data types of each column in the in predictors DataFrame:
Year                   int64
Month                 object
KWH                  float64
Billed $             float64
$/KWH                float64
Water Kgal           float64
KWH/Kgal             float64
E Billed per Kgal    float64
dtype: object


In [6]:
# Convert Month, Day, and Year columns to a datetime format
flow['Date'] = pd.to_datetime(flow['Year'].astype(str) + '-' + flow['Month'].astype(str) + '-' + flow['Date'].astype(str), errors='coerce')

In [7]:
# Create 'Date' column from 'Year' and 'Month'
energy['Date'] = pd.to_datetime(energy['Year'].astype(str) + '-' + energy['Month'] + '-01')

In [8]:
# Check for missing values in each column of flow_data
missing_values = flow.isna().sum()
print("Missing values in each column before removal:")
print(missing_values)

Missing values in each column before removal:
Year                0
Month               0
Date                0
Airport Wells       0
Gal Units1          0
Total Flow1         0
Ft. Bliss Wells     0
Gal Units2          0
Total Flow2         0
Concentrate         0
Gal Units3          0
Total Flow3         0
Finished Water      0
Gal Units4          0
Total Flow4         0
CL2                 0
dtype: int64


In [9]:
# Check for missing values in each column of flow_data
missing_values = energy.isna().sum()
print("Missing values in each column before removal:")
print(missing_values)

Missing values in each column before removal:
Year                 0
Month                0
KWH                  0
Billed $             0
$/KWH                1
Water Kgal           2
KWH/Kgal             2
E Billed per Kgal    3
Date                 0
dtype: int64


In [10]:
# Filling missing values

# For '/KWH' column, we'll fill with the mean
energy['$/KWH'].fillna(energy['$/KWH'].mean(), inplace=True)

# For 'Water Kgal' column, we'll fill with the mean
energy['Water Kgal'].fillna(energy['Water Kgal'].mean(), inplace=True)

# For 'KWH/Kgal' column, we'll fill with the mean
energy['KWH/Kgal'].fillna(energy['KWH/Kgal'].mean(), inplace=True)

# For 'E Billed per Kgal' column, we'll fill with the mean
energy['E Billed per Kgal'].fillna(energy['E Billed per Kgal'].mean(), inplace=True)

# Checking if all missing values are filled
print(energy.isnull().sum())

Year                 0
Month                0
KWH                  0
Billed $             0
$/KWH                0
Water Kgal           0
KWH/Kgal             0
E Billed per Kgal    0
Date                 0
dtype: int64


In [11]:
flow = pd.get_dummies(flow, columns=['Month'], drop_first=True)

In [12]:
energy = pd.get_dummies(energy, columns=['Month'], drop_first=True)

In [13]:
column_data_types = flow.dtypes

# Print the data types
print("Data types of each column in the in predictors DataFrame:")
print(column_data_types)

Data types of each column in the in predictors DataFrame:
Year                         int64
Date                datetime64[ns]
Airport Wells                int64
Gal Units1                   int64
Total Flow1                float64
Ft. Bliss Wells              int64
Gal Units2                   int64
Total Flow2                float64
Concentrate                  int64
Gal Units3                 float64
Total Flow3                float64
Finished Water              object
Gal Units4                   int64
Total Flow4                float64
CL2                        float64
Month_August                 uint8
Month_December               uint8
Month_February               uint8
Month_January                uint8
Month_July                   uint8
Month_June                   uint8
Month_March                  uint8
Month_May                    uint8
Month_November               uint8
Month_October                uint8
Month_September              uint8
dtype: object


In [14]:
column_data_types = energy.dtypes

# Print the data types
print("Data types of each column in the in predictors DataFrame:")
print(column_data_types)

Data types of each column in the in predictors DataFrame:
Year                          int64
KWH                         float64
Billed $                    float64
$/KWH                       float64
Water Kgal                  float64
KWH/Kgal                    float64
E Billed per Kgal           float64
Date                 datetime64[ns]
Month_Aug                     uint8
Month_Dec                     uint8
Month_Feb                     uint8
Month_Feb                     uint8
Month_Jan                     uint8
Month_Jan                     uint8
Month_Jul                     uint8
Month_Jun                     uint8
Month_Mar                     uint8
Month_May                     uint8
Month_Nov                     uint8
Month_Oct                     uint8
Month_Sep                     uint8
dtype: object


In [15]:
def convert_to_numeric(column):
    """Convert a column to numeric, non-numeric becomes NaN."""
    return pd.to_numeric(column, errors='coerce')

# Apply this function to every column in flow
for col in flow.columns:
    if flow[col].dtype == 'object':
        flow[col] = convert_to_numeric(flow[col])

In [16]:
column_data_types = flow.dtypes

# Print the data types
print("Data types of each column in the in predictors DataFrame:")
print(column_data_types)

Data types of each column in the in predictors DataFrame:
Year                         int64
Date                datetime64[ns]
Airport Wells                int64
Gal Units1                   int64
Total Flow1                float64
Ft. Bliss Wells              int64
Gal Units2                   int64
Total Flow2                float64
Concentrate                  int64
Gal Units3                 float64
Total Flow3                float64
Finished Water             float64
Gal Units4                   int64
Total Flow4                float64
CL2                        float64
Month_August                 uint8
Month_December               uint8
Month_February               uint8
Month_January                uint8
Month_July                   uint8
Month_June                   uint8
Month_March                  uint8
Month_May                    uint8
Month_November               uint8
Month_October                uint8
Month_September              uint8
dtype: object


In [17]:
# Check for missing values in each column of flow_data
missing_values = flow.isna().sum()
print("Missing values in each column before removal:")
print(missing_values)

Missing values in each column before removal:
Year                0
Date                0
Airport Wells       0
Gal Units1          0
Total Flow1         0
Ft. Bliss Wells     0
Gal Units2          0
Total Flow2         0
Concentrate         0
Gal Units3          0
Total Flow3         0
Finished Water      1
Gal Units4          0
Total Flow4         0
CL2                 0
Month_August        0
Month_December      0
Month_February      0
Month_January       0
Month_July          0
Month_June          0
Month_March         0
Month_May           0
Month_November      0
Month_October       0
Month_September     0
dtype: int64


In [18]:
# Example: Fill missing values using forward fill method
flow.fillna(method='ffill', inplace=True)

In [19]:
# Check for missing values in each column of flow_data
missing_values = flow.isna().sum()
print("Missing values in each column before removal:")
print(missing_values)

Missing values in each column before removal:
Year                0
Date                0
Airport Wells       0
Gal Units1          0
Total Flow1         0
Ft. Bliss Wells     0
Gal Units2          0
Total Flow2         0
Concentrate         0
Gal Units3          0
Total Flow3         0
Finished Water      0
Gal Units4          0
Total Flow4         0
CL2                 0
Month_August        0
Month_December      0
Month_February      0
Month_January       0
Month_July          0
Month_June          0
Month_March         0
Month_May           0
Month_November      0
Month_October       0
Month_September     0
dtype: int64


In [20]:
print(flow.columns)

Index(['Year', 'Date', 'Airport Wells ', 'Gal Units1', 'Total Flow1',
       'Ft. Bliss Wells ', 'Gal Units2', 'Total Flow2', 'Concentrate',
       'Gal Units3', 'Total Flow3', 'Finished Water', 'Gal Units4',
       'Total Flow4', 'CL2', 'Month_August', 'Month_December',
       'Month_February', 'Month_January', 'Month_July', 'Month_June',
       'Month_March', 'Month_May', 'Month_November', 'Month_October',
       'Month_September'],
      dtype='object')


In [21]:
column_data_types = flow.dtypes

# Print the data types
print("Data types of each column in the in predictors DataFrame:")
print(column_data_types)

Data types of each column in the in predictors DataFrame:
Year                         int64
Date                datetime64[ns]
Airport Wells                int64
Gal Units1                   int64
Total Flow1                float64
Ft. Bliss Wells              int64
Gal Units2                   int64
Total Flow2                float64
Concentrate                  int64
Gal Units3                 float64
Total Flow3                float64
Finished Water             float64
Gal Units4                   int64
Total Flow4                float64
CL2                        float64
Month_August                 uint8
Month_December               uint8
Month_February               uint8
Month_January                uint8
Month_July                   uint8
Month_June                   uint8
Month_March                  uint8
Month_May                    uint8
Month_November               uint8
Month_October                uint8
Month_September              uint8
dtype: object


In [22]:
# Removing outliers beyond 99th percentile
for col in ['KWH']:
    cap = energy[col].quantile(0.99)
    energy = energy[energy[col] <= cap]

In [23]:
# Create lag variables
n_lags = 3

# Create lagged features for "Water Flows"
for i in range(1, n_lags + 1):
    col_name = f'flows_Lag_{i}'
    flow[col_name] = flow['Total Flow4'].shift(i)

# Remove the rows with NaN values generated by shifting
flow.dropna(inplace=True)

# Show the first few rows to confirm
print(flow.head())

   Year       Date  Airport Wells   Gal Units1  Total Flow1  Ft. Bliss Wells   \
3  2018 2018-01-04          773753        7208        7.208            674512   
4  2018 2018-01-05          780945        7192        7.192            676795   
5  2018 2018-01-06          788130        7185        7.185            679076   
6  2018 2018-01-07          795297        7167        7.167            681355   
7  2018 2018-01-08          802438        7141        7.141            683630   

   Gal Units2  Total Flow2  Concentrate  Gal Units3  ...  Month_July  \
3        2277        2.277      5184502      1271.0  ...           0   
4        2283        2.283      5185769      1267.0  ...           0   
5        2281        2.281      5187049      1280.0  ...           0   
6        2279        2.279      5188309      1260.0  ...           0   
7        2275        2.275      5189565      1256.0  ...           0   

   Month_June  Month_March  Month_May  Month_November  Month_October  \
3       

In [24]:
# Create lag variables
n_lags = 3

# Create lagged features for "Energy Consumption"
for i in range(1, n_lags + 1):
    col_name = f'energy_Lag_{i}'
    energy[col_name] = energy['KWH'].shift(i)

# Remove the rows with NaN values generated by shifting
energy.dropna(inplace=True)

# Show the first few rows to confirm
print(energy.head())

   Year       KWH  Billed $     $/KWH  Water Kgal  KWH/Kgal  \
3  2017  769057.0  56103.62  0.072951    217626.0  3.533847   
4  2017  794757.0  58179.68  0.073204    271849.0  2.923524   
5  2017  987154.0  93261.56  0.094475    250467.0  3.941254   
6  2017  850185.0  89740.36  0.105554    255110.0  3.332621   
7  2017  847592.0  89147.44  0.105177    253356.0  3.345459   

   E Billed per Kgal       Date  Month_Aug  Month_Dec  ...  Month_Jul  \
3           0.257798 2017-04-01          0          0  ...          0   
4           0.214015 2017-05-01          0          0  ...          0   
5           0.372351 2017-06-01          0          0  ...          0   
6           0.351771 2017-07-01          0          0  ...          1   
7           0.351866 2017-08-01          1          0  ...          0   

   Month_Jun  Month_Mar  Month_May  Month_Nov  Month_Oct  Month_Sep  \
3          0          0          0          0          0          0   
4          0          0          1      

In [28]:
# Split sizes for Treated Water
train_size = int(len(flow) * 0.5)  # 50% for training

# Train-Test Split
train, test = flow[:train_size], flow[train_size:]

# Prepare data for machine learning models for actual values

# Training data
X_train_actual = train.drop(['Date', 'Total Flow4'], axis=1)
y_train_treated_actual = train['Total Flow4']

# Test data
X_test_actual = test.drop(['Date', 'Total Flow4'], axis=1)
y_test_treated_actual = test['Total Flow4']

print("Training set size:", len(X_train_actual))
print("Test set size:", len(X_test_actual))

Training set size: 889
Test set size: 889


In [25]:
# Split sizes for Energy Consumption
train_size_energy = int(len(energy) * 0.5)  # 50% for training

# Train-Test Split
train_energy, test_energy = energy[:train_size_energy], energy[train_size_energy:]

# Prepare data for machine learning models for actual values

# Training data
X_train_actual_energy = train_energy.drop(['Date', 'KWH'], axis=1)
y_train_actual_energy = train_energy['KWH']

# Test data
X_test_actual_energy = test_energy.drop(['Date', 'KWH'], axis=1)
y_test_actual_energy = test_energy['KWH']

print("Training set size:", len(X_train_actual_energy))
print("Test set size:", len(X_test_actual_energy))


Training set size: 42
Test set size: 43


In [26]:
# Importing required libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from math import sqrt

In [29]:
# Machine Learning Model Training for Treated Water
rf_model_treated = RandomForestRegressor()
rf_model_treated.fit(X_train_actual, y_train_treated_actual)

gb_model_treated = GradientBoostingRegressor()
gb_model_treated.fit(X_train_actual, y_train_treated_actual)

# SARIMA Model Training for Treated Water
try:
    sarima_model_treated = SARIMAX(y_train_treated_actual, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    sarima_fit_treated = sarima_model_treated.fit(disp=False)
except Exception as e:
    print("Error fitting SARIMA model for Treated Water:", str(e))

C:\Users\User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [30]:
# Machine Learning Model Training for Energy Consumption
rf_model_energy = RandomForestRegressor()
rf_model_energy.fit(X_train_actual_energy, y_train_actual_energy)

gb_model_energy = GradientBoostingRegressor()
gb_model_energy.fit(X_train_actual_energy, y_train_actual_energy)


# SARIMA Model Training for Energy Consumption
try:
    sarima_model_energy = SARIMAX(y_train_actual_energy, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    sarima_fit_energy = sarima_model_energy.fit(disp=False)
except Exception as e:
    print("Error fitting SARIMA model for Energy Consumption:", str(e))

C:\Users\User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\User\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


In [31]:
# Predictions using Random Forest
rf_pred_flow = rf_model_treated.predict(X_test_actual)

# Predictions using Gradient Boosting
gb_pred_flow = gb_model_treated.predict(X_test_actual)

# Predictions using SARIMA
start_point_flow = len(y_train_treated_actual)
end_point_flow = start_point_flow + len(X_test_actual) - 1
sarima_pred_flow = sarima_fit_treated.predict(start=start_point_flow, end=end_point_flow)

# Ensemble Predictions for Treated Water
flow_outputs = [rf_pred_flow, gb_pred_flow, sarima_pred_flow]
ensemble_pred_flow = np.mean(flow_outputs, axis=0)


C:\Users\User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [32]:
# Predictions using Random Forest
rf_pred_energy = rf_model_energy.predict(X_test_actual_energy)

# Predictions using Gradient Boosting
gb_pred_energy = gb_model_energy.predict(X_test_actual_energy)

# Predictions using SARIMA Model
start_point_energy = len(y_train_actual_energy)
end_point_energy = start_point_energy + len(X_test_actual_energy) - 1
sarima_pred_energy = sarima_fit_energy.predict(start=start_point_energy, end=end_point_energy)

# Ensemble Predictions for Energy Consumption
energy_outputs = [rf_pred_energy, gb_pred_energy, sarima_pred_energy]
ensemble_pred_energy = np.mean(energy_outputs, axis=0)


C:\Users\User\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [33]:
def error(y_true, y_pred, threshold=0.5):
    """
    Calculate the error rate based on a given threshold.
    The function returns the proportion of predictions that are off by more than the threshold.
    """
    return np.mean(np.abs(y_true - y_pred) > threshold)

In [34]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
import numpy as np

def evaluate_predictions(y_true, y_pred, model_name):
    """
    Evaluate and print RMSE, MAE, and Error Rate for a given model's predictions.
    """
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    err_rate = error(y_true, y_pred)
    
    print(f"=== {model_name} ===")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"Error Rate: {err_rate:.4f}\n")
    return rmse, mae, err_rate

def main():
    # Evaluating Water Flow Predictions
    print("Evaluations for Water Flow:\n")
    evaluate_predictions(y_test_treated_actual, rf_pred_flow, "Random Forest")
    evaluate_predictions(y_test_treated_actual, gb_pred_flow, "Gradient Boosting")
    evaluate_predictions(y_test_treated_actual, sarima_pred_flow, "SARIMA")
    evaluate_predictions(y_test_treated_actual, ensemble_pred_flow, "Ensemble")
    
    # Evaluating Energy Consumption Predictions
    print("Evaluations for Energy Consumption:\n")
    evaluate_predictions(y_test_actual_energy, rf_pred_energy, "Random Forest")
    evaluate_predictions(y_test_actual_energy, gb_pred_energy, "Gradient Boosting")
    evaluate_predictions(y_test_actual_energy, sarima_pred_energy, "SARIMA")
    evaluate_predictions(y_test_actual_energy, ensemble_pred_energy, "Ensemble")

# Run the evaluations
main()

Evaluations for Water Flow:

=== Random Forest ===
RMSE: 1.2590
MAE: 0.4772
Error Rate: 0.1507

=== Gradient Boosting ===
RMSE: 0.9529
MAE: 0.3934
Error Rate: 0.2070

=== SARIMA ===
RMSE: 6.4837
MAE: 5.5620
Error Rate: 0.9395

=== Ensemble ===
RMSE: 2.6108
MAE: 2.0241
Error Rate: 0.8695

Evaluations for Energy Consumption:

=== Random Forest ===
RMSE: 665443.2633
MAE: 550343.4792
Error Rate: 1.0000

=== Gradient Boosting ===
RMSE: 616283.7353
MAE: 504401.1845
Error Rate: 1.0000

=== SARIMA ===
RMSE: 759367.7005
MAE: 555713.9907
Error Rate: 0.8298

=== Ensemble ===
RMSE: 607133.6043
MAE: 494003.6671
Error Rate: 1.0000

